### Metric 1: Sprint Context Quality
 
This notebook detects sprints from raw tracking data and enriches them with
tactical context from phases of play. Goal is to separate high-volume sprinters
from players whose sprints occur in tactically valuable situations.

## Data Sources

- **Tracking data:** Raw position data at 10fps (JSONL files per match)
- **Phases of play:** Preprocessed CSV with tactical context
- **Player metadata:** Preprocessed CSV with minutes played and positions

## Approach

1. Detect discrete sprint events from tracking (speed > 25 km/h)
2. Join each sprint to its phase context using mid-frame
3. Flag high-value contexts (create/finish phases, shot possessions)
4. Aggregate to player level using PySpark
5. Calculate per-90 metrics and filter minimum minutes

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sys
sys.path.append('..')
from src.loaders import get_all_match_ids, load_tracking_data, load_match_metadata
from src.metrics import (
    detect_sprints,
    enrich_sprints_with_phases,
    add_sprint_context_flags,
    aggregate_player_sprints,
)

# Initialize Spark session
spark = SparkSession.builder \
    .appName("sprint-context-quality") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/10 15:04:54 WARN Utils: Your hostname, Anguss-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.4.158 instead (on interface en0)
25/12/10 15:04:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 15:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1


In [2]:
# Load preprocessed data
data_dir = Path('../output')

all_phases = pd.read_csv(data_dir / 'all_phases.csv')
player_metadata = pd.read_csv(data_dir / 'player_metadata.csv')

print(f"Loaded {len(all_phases):,} phase records")
print(f"Loaded {len(player_metadata):,} player-match records")

unique_players = player_metadata['player_id'].nunique()
print("Unique players in dataset:", unique_players)

Loaded 4,581 phase records
Loaded 360 player-match records
Unique players in dataset: 237


## Step 1: Detect Sprints from Tracking Data

Loop through each match, load tracking data, detect sprints.
This is the most time-consuming step (~2-3 min for 10 matches).

In [3]:
match_ids = get_all_match_ids()
print(f"Processing {len(match_ids)} matches...")

sprints_list = []

for i, match_id in enumerate(match_ids, 1):
    print(f"[{i}/{len(match_ids)}] Detecting sprints in match {match_id}...", end=' ')
    
    try:
        tracking = load_tracking_data(match_id)
        metadata = load_match_metadata(match_id)
        
        match_sprints = detect_sprints(tracking, match_id)
        sprints_list.append(match_sprints)
        
        print(f"✓ {len(match_sprints)} sprints detected")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        continue

# Combine all matches
sprints_df = pd.concat(sprints_list, ignore_index=True)

print(f"\nTotal sprints detected: {len(sprints_df):,}")
print(f"Unique players: {sprints_df['player_id'].nunique()}")

Processing 10 matches...
[1/10] Detecting sprints in match 2017461... ✓ 158 sprints detected
[2/10] Detecting sprints in match 2015213... ✓ 211 sprints detected
[3/10] Detecting sprints in match 2013725... ✓ 206 sprints detected
[4/10] Detecting sprints in match 2011166... ✓ 194 sprints detected
[5/10] Detecting sprints in match 2006229... ✓ 118 sprints detected
[6/10] Detecting sprints in match 1996435... ✓ 156 sprints detected
[7/10] Detecting sprints in match 1953632... ✓ 102 sprints detected
[8/10] Detecting sprints in match 1925299... ✓ 72 sprints detected
[9/10] Detecting sprints in match 1899585... ✓ 153 sprints detected
[10/10] Detecting sprints in match 1886347... ✓ 151 sprints detected

Total sprints detected: 1,521
Unique players: 180


In [4]:
# Sprint duration summary
print("Sprint duration distribution (rounded):")
print(sprints_df["duration_s"].describe().round(2))

# Sprint avg speed summary
print("\nSprint avg speed distribution (rounded):")
print(sprints_df["avg_sprint_speed_kmh"].describe().round(2))

# Sample sprints (rounded)
print("\nSample sprints (rounded):")

sample_cols = [
    "match_id", "player_id", "frame_start", "frame_end",
    "duration_s", "distance_m", "avg_sprint_speed_kmh"
]

sample = sprints_df[sample_cols].head(10).copy()

# Round only the numeric columns
for col in ["duration_s", "distance_m", "avg_sprint_speed_kmh"]:
    sample[col] = sample[col].round(2)

display(sample)

Sprint duration distribution (rounded):
count    1521.00
mean        2.62
std         1.27
min         0.90
25%         1.70
50%         2.30
75%         3.30
max         9.20
Name: duration_s, dtype: float64

Sprint avg speed distribution (rounded):
count    1521.00
mean       26.78
std         0.91
min        25.24
25%        26.00
50%        26.63
75%        27.46
max        28.97
Name: avg_sprint_speed_kmh, dtype: float64

Sample sprints (rounded):


,match_id,player_id,frame_start,frame_end,duration_s,distance_m,avg_sprint_speed_kmh
0,2017461,4322,2641,2666,2.6,20.03,27.73
1,2017461,4322,10680,10734,5.5,43.19,28.27
2,2017461,4322,15820,15832,1.3,9.64,26.68
3,2017461,4322,20298,20317,2.0,14.65,26.37
4,2017461,4322,54454,54463,1.0,7.37,26.54
5,2017461,4322,60691,60736,4.6,35.74,27.97
6,2017461,11117,5896,5921,2.6,20.68,28.63
7,2017461,11117,9817,9831,1.5,10.84,26.00
8,2017461,11117,10362,10381,2.0,15.21,27.39
9,2017461,11117,15300,15341,4.2,31.31,26.84


## Step 2: Enrich Sprints with Phase Context

Join each sprint to its corresponding phase using the sprint's mid-frame.
This adds tactical context like possession phase type and outcome flags.

In [5]:
# Make sure match_id is string for joins
sprints_df['match_id'] = sprints_df['match_id'].astype(str)
all_phases['match_id'] = all_phases['match_id'].astype(str)
player_metadata['match_id'] = player_metadata['match_id'].astype(str)

sprints_enriched = enrich_sprints_with_phases(sprints_df, all_phases)

# Check phase join coverage
phase_coverage = sprints_enriched['team_in_possession_phase_type'].notna().mean()
print(f"Phase join coverage: {phase_coverage:.1%}")

print("\nPhase type distribution:")
print(sprints_enriched['team_in_possession_phase_type'].value_counts())

Phase join coverage: 91.9%

Phase type distribution:
team_in_possession_phase_type
create         464
transition     233
finish         227
build_up       138
chaotic        138
direct         110
set_play        46
quick_break     42
Name: count, dtype: int64


In [6]:
# Attach team_id to each sprint (needed for attacking/defensive classification)
sprints_enriched = sprints_enriched.merge(
    player_metadata[['match_id', 'player_id', 'team_id']].drop_duplicates(),
    on=['match_id', 'player_id'],
    how='left'
)

# Add attacking/defensive + spatial context flags
sprints_enriched = add_sprint_context_flags(sprints_enriched)

# Quick inspection of key fields to ensure joins/flags look correct
display(
    sprints_enriched[
        [
            'match_id', 'player_id', 'team_id',
            'team_in_possession_id',
            'team_in_possession_phase_type',
            'is_attacking_sprint',
            'is_high_value_phase'
        ]
    ].head(10)
)

# Summary of context classifications
print("Context flags added:")
print(f"- High-value phases: {sprints_enriched['is_high_value_phase'].mean():.1%}")
print(f"- Attacking sprints: {sprints_enriched['is_attacking_sprint'].mean():.1%}")
print(f"- Defensive sprints: {sprints_enriched['is_defensive_sprint'].mean():.1%}")

,match_id,player_id,team_id,team_in_possession_id,team_in_possession_phase_type,is_attacking_sprint,is_high_value_phase
0,2017461,4322,4177,868.0,finish,False,True
1,2017461,4322,4177,4177.0,create,True,True
2,2017461,4322,4177,NaN,None,False,False
3,2017461,4322,4177,4177.0,quick_break,True,True
4,2017461,4322,4177,868.0,finish,False,True
5,2017461,4322,4177,868.0,transition,False,True
6,2017461,11117,868,868.0,finish,True,True
7,2017461,11117,868,4177.0,create,False,True
8,2017461,11117,868,NaN,None,False,False
9,2017461,11117,868,868.0,create,True,True


Context flags added:
- High-value phases: 63.5%
- Attacking sprints: 41.4%
- Defensive sprints: 58.6%


## Step 3: Aggregate to Player Level using PySpark

Convert to Spark DataFrames and aggregate sprint metrics per player per match.

In [7]:
# Convert to Spark DataFrames
sprints_sdf = spark.createDataFrame(sprints_enriched)
player_meta_sdf = spark.createDataFrame(player_metadata)

print(f"Sprints in Spark: {sprints_sdf.count():,} rows")
print(f"Player metadata in Spark: {player_meta_sdf.count():,} rows")

Sprints in Spark: 1,521 rows
Player metadata in Spark: 360 rows


25/12/10 15:05:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
# Aggregate using PySpark
player_sprints_sdf = aggregate_player_sprints(
    sprints_sdf, 
    player_meta_sdf,
    min_minutes=20.0
)

print(f"Player-match records after aggregation: {player_sprints_sdf.count()}")

Player-match records after aggregation: 244


In [9]:
# Convert to pandas
player_sprints = player_sprints_sdf.toPandas()

print(f"Final dataset: {len(player_sprints)} player-match records")
print(f"Unique players: {player_sprints['player_id'].nunique()}")

# Save clean metrics (no metadata)
sprint_cols = [
    'match_id', 'player_id',
    'sprint_count', 'sprints_per_90', 'sprint_distance_m', 'sprint_distance_per_90',
    'avg_sprint_speed_kmh', 'max_sprint_speed_kmh',
    'high_value_sprint_pct', 'attacking_sprint_pct', 'defensive_sprint_pct',
    'sprints_in_shot_possessions_pct', 'sprints_in_goal_possessions_pct',
    'sprints_in_attacking_third_pct',
    'high_value_sprints_per_90',
]

player_sprints[sprint_cols].to_csv('../output/player_sprints.csv', index=False)
print(f"\nSaved {len(sprint_cols)} metric columns to ../output/player_sprints.csv")
print(f"player_sprints still has all {len(player_sprints.columns)} columns for analysis")

Final dataset: 244 player-match records
Unique players: 166

Saved 15 metric columns to ../output/player_sprints.csv
player_sprints still has all 21 columns for analysis


## Results: Sprint Context Quality Rankings

Different ways to rank players based on sprint volume, context, and outcomes.

### Highest Sprint Volume (per 90)

This table highlights players who produce the most sprint actions relative to their playing time.  
Sprint volume varies heavily by role. As expected, wide attackers, centre forwards, and high-mobility midfielders appear frequently in the top results. A small number of full backs and centre backs also appear, reflecting matches where they recorded unusually high sprint involvement.

Values are normalised to a per-90 rate to enable fair comparison across players with different minutes played.

In [10]:
# Top 10 by sprint volume
print("=== Highest Sprint Volume (per 90) ===\n")

top_volume = player_sprints.nlargest(15, "sprints_per_90")[
    ["player_short_name", "position_group", "minutes_played",
     "sprint_count", "sprints_per_90", "high_value_sprint_pct"]
]

# Round all numeric columns to 2dp for display
top_volume_rounded = top_volume.copy()
num_cols = top_volume_rounded.select_dtypes("number").columns
top_volume_rounded[num_cols] = top_volume_rounded[num_cols].round(2)

print(top_volume_rounded.to_string(index=False))

=== Highest Sprint Volume (per 90) ===

player_short_name   position_group  minutes_played  sprint_count  sprints_per_90  high_value_sprint_pct
       J. Randall   Center Forward           27.58             9           29.37                   0.44
       L. Gillion    Wide Attacker           33.90             7           18.58                   0.71
  L. Brooke-Smith         Midfield           40.13             8           17.94                   0.62
          M. Ruhs   Center Forward           71.22            14           17.69                   0.71
        B. Hamill Central Defender           30.76             6           17.56                   0.33
        J. Lauton         Midfield           51.78            10           17.38                   0.80
         C. Piper         Midfield           98.82            19           17.30                   0.58
       J. Randall   Center Forward           63.52            12           17.00                   0.67
       A. Segecic       

### Best Sprint Context Quality (% High-Value Sprints)

Here we evaluate not just how often players sprint, but **where those sprints occur within possession structure**.  
High-value phases include `create`, `finish`, `transition`, and `quick_break` - moments most likely to produce threat.

Players scoring highly here consistently time their high-intensity actions during meaningful attacking moments.  
Some players show 100% because:
- they appear in only 1–2 matches in this sample, and  
- their limited sprint actions all occurred during attacking or transitional phases, which is plausible for certain roles (wingers, forwards, overlapping full backs).

### Note on sprint thresholds  
Players must have **at least 5 sprints** to appear in context-quality leaderboards.  
Percentages such as *% high-value sprints* or *% linked to shot possessions* are unreliable with very small samples, so this cutoff removes noise while keeping comparisons meaningful.

In [11]:
# Top 10 by context quality
print("=== Best Sprint Context Quality ===")
print("(Players with >5 sprints, sorted by high-value %)\n")

quality_df = player_sprints[player_sprints["sprint_count"] >= 5]

top_quality = quality_df.nlargest(15, "high_value_sprint_pct")[
    ["player_short_name", "position_group", "sprint_count", "sprints_per_90",
     "high_value_sprint_pct", "sprints_in_shot_possessions_pct"]
]

top_quality_rounded = top_quality.copy()
num_cols = top_quality_rounded.select_dtypes("number").columns
top_quality_rounded[num_cols] = top_quality_rounded[num_cols].round(2)

print(top_quality_rounded.to_string(index=False))

=== Best Sprint Context Quality ===
(Players with >5 sprints, sorted by high-value %)

player_short_name   position_group  sprint_count  sprints_per_90  high_value_sprint_pct  sprints_in_shot_possessions_pct
        R. Teague         Midfield             6            5.54                   1.00                             0.17
        N. Vergos   Center Forward             5            5.41                   1.00                             0.60
       J. Hollman         Midfield             6            8.32                   1.00                             0.00
        M. Leckie         Midfield            10           10.45                   1.00                             0.30
      M. Sheridan Central Defender             5            4.46                   1.00                             0.60
       L. Bayliss         Midfield             6            8.77                   1.00                             0.00
      F. De Vries        Full Back             5            4.57  

### Sprints Most Linked to Shot Possessions

This table identifies players whose sprint actions frequently appear within possessions that end in shots.

A high proportion suggests:
- strong involvement in chance creation,  
- effective movement off the ball,  
- or defensive sprints that directly precede regains leading to shots.

Threshold: only players with ≥5 sprints are included to keep the metric stable.

In [12]:
# Top 10 by outcome linkage
print("=== Sprints Most Linked to Shot Possessions ===")
print("(Players with >5 sprints)\n")

outcome_df = player_sprints[player_sprints["sprint_count"] >= 5]

top_outcome = outcome_df.nlargest(10, "sprints_in_shot_possessions_pct")[
    ["player_short_name", "position_group", "sprint_count",
     "sprints_in_shot_possessions_pct", "high_value_sprint_pct"]
]

top_outcome_rounded = top_outcome.copy()
num_cols = top_outcome_rounded.select_dtypes("number").columns
top_outcome_rounded[num_cols] = top_outcome_rounded[num_cols].round(2)

print(top_outcome_rounded.to_string(index=False))

=== Sprints Most Linked to Shot Possessions ===
(Players with >5 sprints)

  player_short_name   position_group  sprint_count  sprints_in_shot_possessions_pct  high_value_sprint_pct
J. Courtney-Perkins Central Defender             6                             0.67                   0.83
          N. Vergos   Center Forward             5                             0.60                   1.00
        M. Sheridan Central Defender             5                             0.60                   1.00
         A. Cáceres         Midfield             9                             0.44                   0.56
          J. Lauton         Midfield            10                             0.40                   0.80
            J. Yull    Wide Attacker            10                             0.40                   0.80
            K. Shaw Central Defender             5                             0.40                   0.80
         A. Segecic         Midfield             5                   

### Best Combined Sprint Profile (Volume × Quality)

This metric blends workload and impact by ranking players with:
- a high number of sprints per 90, and  
- a high share of those sprints occurring in high-value phases.

It rewards players who both *work often* and *work at the right moments*, surfacing attackers and full backs who consistently contribute to meaningful ball-advancing actions.

In [13]:
# Top 10 by combined volume + quality
print("=== Best Combined Volume + Quality ===")
print("(High-value sprints per 90)\n")

top_combined = player_sprints.nlargest(10, "high_value_sprints_per_90")[
    ["player_short_name", "position_group", "minutes_played",
     "sprints_per_90", "high_value_sprint_pct", "high_value_sprints_per_90"]
]

top_combined_rounded = top_combined.copy()
num_cols = top_combined_rounded.select_dtypes("number").columns
top_combined_rounded[num_cols] = top_combined_rounded[num_cols].round(2)

print(top_combined_rounded.to_string(index=False))

=== Best Combined Volume + Quality ===
(High-value sprints per 90)

player_short_name position_group  minutes_played  sprints_per_90  high_value_sprint_pct  high_value_sprints_per_90
        Y. Dukuly  Wide Attacker           23.55           15.29                   1.00                      15.29
        J. Lauton       Midfield           51.78           17.38                   0.80                      13.90
      B. Garuccio      Full Back           94.65           15.21                   0.88                      13.31
       L. Gillion  Wide Attacker           33.90           18.58                   0.71                      13.27
      M. Francois  Wide Attacker           48.05           14.98                   0.88                      13.11
       J. Randall Center Forward           27.58           29.37                   0.44                      13.05
          M. Ruhs Center Forward           71.22           17.69                   0.71                      12.64
        C. T

### Sprint Metrics by Position Group

Sprint behaviour varies significantly by role, so aggregating by position group provides important context.  
Typical patterns emerge:

- Wide attackers & full backs → highest sprint volume  
- Midfielders → balanced profiles  
- Centre backs → lowest sprint volume but meaningful defensive sprint contributions  

This validates that sprint detection aligns with expected positional demands.

In [14]:
# Position group breakdown
print("=== Sprint Volume by Position ===\n")
position_summary = player_sprints.groupby('position_group')['sprints_per_90'].agg([
    'count', 'mean', 'median', 'min', 'max'
]).round(2)
print(position_summary)

=== Sprint Volume by Position ===

                  count  mean  median   min    max
position_group                                    
Center Forward       41  9.01    7.98  1.39  29.37
Central Defender     45  5.46    5.46  0.88  17.56
Full Back            42  8.55    8.23  0.88  16.16
Midfield             61  6.03    5.21  0.88  17.94
Wide Attacker        55  9.09    8.28  1.78  18.58


In [15]:
# Check aggregation structure
print("=== Aggregation Check ===")
print(f"Total player-match records: {len(player_sprints)}")
print(f"Unique players: {player_sprints['player_id'].nunique()}")
print("Records per player distribution:")
print(player_sprints.groupby("player_id").size().value_counts().head())

# Check phase distributions
print("\n=== Phase Distribution in Raw Sprints ===")
print(
    sprints_enriched["is_high_value_phase"]
    .value_counts(normalize=True)
    .round(3)
)

# Check sprint counts make sense
print("\n=== Sprint Counts ===")
print(
    f"Median sprints per player-match: "
    f"{player_sprints['sprint_count'].median():.2f}"
)
print(
    f"Median sprints per 90: "
    f"{player_sprints['sprints_per_90'].median():.2f}"
)

print("\n=== Phase Type Distribution ===")
print(all_phases['team_in_possession_phase_type'].value_counts(normalize=True))

=== Aggregation Check ===
Total player-match records: 244
Unique players: 166
Records per player distribution:
1    106
2     49
4      7
3      4
Name: count, dtype: int64

=== Phase Distribution in Raw Sprints ===
is_high_value_phase
True     0.635
False    0.365
Name: proportion, dtype: float64

=== Sprint Counts ===
Median sprints per player-match: 5.00
Median sprints per 90: 6.59

=== Phase Type Distribution ===
team_in_possession_phase_type
create         0.318489
chaotic        0.237284
finish         0.160445
build_up       0.134905
direct         0.081423
set_play       0.037983
transition     0.017027
quick_break    0.012443
Name: proportion, dtype: float64


## Validation Notes

**Sprint detection**

- Median of ~6–7 sprints per 90 across all positions using a ≥25.2 km/h sprint threshold  
- Wide attackers and full backs show the highest volumes (often 10–15 sprints per 90), central defenders the lowest (3–6), which matches typical physical reports for professional football

**Context quality**

- High-value phases (`create`, `finish`, `quick_break`, `transition`) make up ~51% of all phases in the phase file  
- 63–64% of detected sprints fall into these phases, confirming that sprints cluster in attacking and transition moments rather than slow build-up
- Some players show **100% of their sprints in high-value phases**. This is still plausible because:
  - most players only appear in **1–2 matches** in this 10-match sample, so they often have only 5–10 detected sprints, and
  - certain roles (centre forwards, wide attackers, overlapping full backs) tend to sprint almost exclusively when their team is creating or finishing attacks; their recovery and positional runs typically sit below the sprint threshold
- The remaining ~36% of sprints occur during `build_up`, `chaotic`, `direct` or `set_play` phases and capture defensive and out-of-possession work

**Speed characteristics**

- Average sprint speeds after smoothing sit around **26–29 km/h**, with 90th-percentile peaks around **30–31 km/h**
- These values are consistent with SkillCorner physical aggregates and published tracking studies when using 10 Hz tracking and a ≥25.2 km/h sprint definition

## Summary Statistics

Overall patterns across all players.

In [16]:
# Position group comparison
print("=== Sprint Metrics by Position Group ===\n")

position_summary = player_sprints.groupby('position_group').agg({
    'sprint_count': 'sum',
    'sprints_per_90': 'mean',
    'high_value_sprint_pct': 'mean',
    'attacking_sprint_pct': 'mean',
    'sprints_in_shot_possessions_pct': 'mean',
}).round(2)

print(position_summary)

=== Sprint Metrics by Position Group ===

                  sprint_count  sprints_per_90  high_value_sprint_pct  \
position_group                                                          
Center Forward             251            9.01                   0.53   
Central Defender           230            5.46                   0.57   
Full Back                  356            8.55                   0.70   
Midfield                   287            6.03                   0.66   
Wide Attacker              359            9.09                   0.67   

                  attacking_sprint_pct  sprints_in_shot_possessions_pct  
position_group                                                           
Center Forward                    0.51                             0.12  
Central Defender                  0.17                             0.23  
Full Back                         0.47                             0.17  
Midfield                          0.29                             0.19  
Wi

## Key findings

- About **64% of sprints** happen in create/finish/transition/quick_break phases, which makes sense – players tend to sprint more when their team is attacking.
- Sprint speeds sit mostly in the **26–29 km/h** range, with peaks around **30–31 km/h** after smoothing, which lines up with SkillCorner’s PSV99 benchmarks.
- Wide attackers and full backs show the **highest sprint volume per 90**, while central defenders are lower, which matches normal positional profiles.